# Main Script

STATIC VERSION
- Given a fixed DoD
- Main function
- Auxillary functions:
    - initial solution creator: fill up vehicles while respecting capacity
    - (greedy) insertion heuristic: in stead of looking for all possible insertions, look for a better insertion until you find no better one
        - initial solution creator > spatio-temporal clustering of requests?
        - feasibility check
        - incremental waiting time + travel time calculator
    - improvement heuristic
        - neighborhood creators (destroy&repair, swappers, L-opt moves, ...)
        - evaluate objective functions

## Todo (update 4/2)

- Test basic network loading functions for larger instances!
- Let non-taken requests 'spill over' into a new request group > create functions
- In the current initial solution, (1,3) travellers benefit a lot <> other travellers: so maybe put a limit on how many travellers can be taken by those
- Some vehicles can drive all the way to 5 (1 > 5 > 1), others return from 3 directly back (1 > 3 > 1)
    - make it that some vehicles are available at the depot: these can be schedules for new rides
- Create travel time evaluators

In [1]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt

import od_traveltime_generation as od
import requests_generation as rg
import solution_generation as sg

In [2]:
# PARAMETERS

#network
network_size = 'small'
interstop_distance = 'half'
v_mean = 50 #km/h

#requests
demand_scenario = 2
time_of_day = 1 #1 = peak, 0 = off-peak
peak_duration = 60 #min.
req_max_cluster_time = 5 #min.

#vehicles
available_vehicles = 12
vehicle_capacity = 20
dod = 0   #static version = 0, dynamic version > 0

#objective function
WT_pen = 3 
TT_pen = 1 

## Load network

In [3]:
# network
network = od.import_network(network_size, interstop_distance)
od_matrix = od.generate_tt_od(network, v_mean)
#od_matrix

## Import requests

In [4]:
lambdapeak = rg.get_scenario_mean_demand('city', network_size, scen=demand_scenario, peak=1)
mupeak = rg.get_scenario_mean_demand('terminal', network_size, scen=demand_scenario, peak=1)

In [5]:
dict_requests = rg.convert_md_todict(lambdapeak, mupeak, demand_scenario)
total_requests = rg.generate_requests(dict_requests, peak_duration)
od_pairs = set(total_requests.keys())

In [6]:
list_all_requests = rg.list_all_requests(total_requests)
list_all_requests

[((1, 2), 0, 0),
 ((1, 2), 1.5020453451562124, 0),
 ((1, 2), 1.6429435260831582, 0),
 ((1, 2), 16.16381613204888, 0),
 ((1, 2), 23.228462377541025, 0),
 ((1, 2), 44.21821412167385, 0),
 ((1, 2), 54.69407627151028, 0),
 ((1, 2), 56.33134879621105, 0),
 ((1, 2), 59.83471037278398, 0),
 ((1, 3), 0, 0),
 ((1, 3), 0.0031443002354868566, 0),
 ((1, 3), 0.09444781351133584, 0),
 ((1, 3), 0.4390512169462489, 0),
 ((1, 3), 0.6782417249864062, 0),
 ((1, 3), 0.9581931116750013, 0),
 ((1, 3), 1.220855845025892, 0),
 ((1, 3), 1.229550791724035, 0),
 ((1, 3), 1.4703887130940334, 0),
 ((1, 3), 1.5471928176834542, 0),
 ((1, 3), 1.6254924213576174, 0),
 ((1, 3), 1.7286073760351315, 0),
 ((1, 3), 1.799794431461125, 0),
 ((1, 3), 2.6109364167279723, 0),
 ((1, 3), 2.798545484790216, 0),
 ((1, 3), 2.986904158549234, 0),
 ((1, 3), 3.099923475862279, 0),
 ((1, 3), 3.263092152195045, 0),
 ((1, 3), 3.68066868812201, 0),
 ((1, 3), 3.9267991299229705, 0),
 ((1, 3), 4.07568466372391, 0),
 ((1, 3), 4.16897635587940

In [7]:
copy_list_requests = list_all_requests.copy()
grouped_requests = rg.group_requests_dt(copy_list_requests, req_max_cluster_time, od_pairs)
grouped_requests

{(2,
  3): [[((2, 3), 0, 0),
   ((2, 3), 0.13582901044301, 0),
   ((2, 3), 0.1677889767868189, 0),
   ((2, 3), 0.3528464529680906, 0),
   ((2, 3), 0.954269843129492, 0),
   ((2, 3), 0.9915945444783486, 0),
   ((2, 3), 1.240312634408136, 0),
   ((2, 3), 1.2527886181925887, 0),
   ((2, 3), 1.3010323892460975, 0),
   ((2, 3), 1.4503403783962279, 0),
   ((2, 3), 1.6585803914259265, 0),
   ((2, 3), 2.095196284097822, 0),
   ((2, 3), 2.1166584343133885, 0),
   ((2, 3), 2.119800901079343, 0),
   ((2, 3), 2.311961444915684, 0),
   ((2, 3), 2.429673400887397, 0),
   ((2, 3), 2.702042404895464, 0),
   ((2, 3), 3.331469795873213, 0),
   ((2, 3), 3.514238217108862, 0),
   ((2, 3), 3.518590017849716, 0),
   ((2, 3), 3.5209704098716217, 0),
   ((2, 3), 3.569672719363702, 0),
   ((2, 3), 3.7784036059315755, 0),
   ((2, 3), 3.887000270549554, 0),
   ((2, 3), 3.8890213588872005, 0),
   ((2, 3), 4.0747685970484335, 0),
   ((2, 3), 4.07654246457662, 0),
   ((2, 3), 4.126692815214457, 0),
   ((2, 3), 4.42

In [8]:
# this shows how big the groups of passengers are (per threshold time from the first departure) + how many there are
count_groups = rg.request_groups_per_od(grouped_requests)
count_groups

{(2, 3): [30, 30, 14, 14, 23, 19, 21, 23, 25, 15, 24, 19],
 (1, 3): [23, 23, 22, 21, 13, 18, 24, 19, 21, 16, 16, 12],
 (4, 5): [1],
 (1, 2): [3, 1, 1, 1, 2, 1],
 (3, 5): [1, 2, 2, 2, 1, 2],
 (3, 4): [1, 1, 1, 2, 1]}

Insertion strategy: momentarily focussed on the capacity constraint (i.e. trying to fill vehicles), not so much on the time constraint

1. Divide requests into chunks: 
 * (a) of at most equal size (e.g. 10) 
 * (b) group requests as long as the maximum waiting time is e.g. 5 min.
2. Select a first chunk of travellers, who want to travel from the origin to the farthest point, and fill a bus with them (almost) 
    * If this bus is ride is full, then this schedule is complete.
    * If there is room left, then add passengers in between at each visited stop.


In [9]:
tocity_keys = [(1,2),(2,3),(1,3)]
toterminal_keys = [(3,4),(3,5),(4,5)]

filterByKey = lambda keys: {x: grouped_requests[x] for x in keys}
tocity_requests = filterByKey(tocity_keys)
toterminal_requests = filterByKey(toterminal_keys)

tocity_requests

{(1,
  2): [[((1, 2), 0, 0),
   ((1, 2), 1.5020453451562124, 0),
   ((1, 2), 1.6429435260831582, 0)], [((1, 2),
    16.16381613204888,
    0)], [((1, 2), 23.228462377541025, 0)], [((1, 2),
    44.21821412167385,
    0)], [((1, 2), 54.69407627151028, 0),
   ((1, 2), 56.33134879621105, 0)], [((1, 2), 59.83471037278398, 0)]],
 (2,
  3): [[((2, 3), 0, 0),
   ((2, 3), 0.13582901044301, 0),
   ((2, 3), 0.1677889767868189, 0),
   ((2, 3), 0.3528464529680906, 0),
   ((2, 3), 0.954269843129492, 0),
   ((2, 3), 0.9915945444783486, 0),
   ((2, 3), 1.240312634408136, 0),
   ((2, 3), 1.2527886181925887, 0),
   ((2, 3), 1.3010323892460975, 0),
   ((2, 3), 1.4503403783962279, 0),
   ((2, 3), 1.6585803914259265, 0),
   ((2, 3), 2.095196284097822, 0),
   ((2, 3), 2.1166584343133885, 0),
   ((2, 3), 2.119800901079343, 0),
   ((2, 3), 2.311961444915684, 0),
   ((2, 3), 2.429673400887397, 0),
   ((2, 3), 2.702042404895464, 0),
   ((2, 3), 3.331469795873213, 0),
   ((2, 3), 3.514238217108862, 0),
   ((2, 3

In [10]:
first_stop = 1
last_stop = 3
first_veh_index = 1

copy_tocity_requests = tocity_requests.copy()

In [11]:
s = sg.create_initial_solution(copy_tocity_requests, first_stop, last_stop, first_veh_index, available_vehicles, vehicle_capacity)

In [12]:
s

{1: {1: [3.9267991299229705,
   [((1, 3), 0, 0),
    ((1, 3), 0.0031443002354868566, 0),
    ((1, 3), 0.09444781351133584, 0),
    ((1, 3), 0.4390512169462489, 0),
    ((1, 3), 0.6782417249864062, 0),
    ((1, 3), 0.9581931116750013, 0),
    ((1, 3), 1.220855845025892, 0),
    ((1, 3), 1.229550791724035, 0),
    ((1, 3), 1.4703887130940334, 0),
    ((1, 3), 1.5471928176834542, 0),
    ((1, 3), 1.6254924213576174, 0),
    ((1, 3), 1.7286073760351315, 0),
    ((1, 3), 1.799794431461125, 0),
    ((1, 3), 2.6109364167279723, 0),
    ((1, 3), 2.798545484790216, 0),
    ((1, 3), 2.986904158549234, 0),
    ((1, 3), 3.099923475862279, 0),
    ((1, 3), 3.263092152195045, 0),
    ((1, 3), 3.68066868812201, 0),
    ((1, 3), 3.9267991299229705, 0)]],
  2: [3.9267991299229705,
   [((1, 3), 0, 0),
    ((1, 3), 0.0031443002354868566, 0),
    ((1, 3), 0.09444781351133584, 0),
    ((1, 3), 0.4390512169462489, 0),
    ((1, 3), 0.6782417249864062, 0),
    ((1, 3), 0.9581931116750013, 0),
    ((1, 3), 1.2